# Day 09 - Deepseek R1 Lite

In [4]:
def compact_disk(disk):
    leftmost_free_index = 0
    rightmost_file_index = len(disk) - 1
    
    while leftmost_free_index < rightmost_file_index:
        # Find the leftmost free space
        while leftmost_free_index < len(disk) and disk[leftmost_free_index] != '.':
            leftmost_free_index += 1
        
        # Find the rightmost file block
        while rightmost_file_index >= 0 and disk[rightmost_file_index] == '.':
            rightmost_file_index -= 1
        
        if leftmost_free_index < rightmost_file_index:
            # Move the file block to the free space
            disk[leftmost_free_index] = disk[rightmost_file_index]
            disk[rightmost_file_index] = '.'
    
    return disk

def calculate_checksum(disk):
    return sum(pos * block for pos, block in enumerate(disk) if block != '.')

def main():
    with open('input.txt', 'r') as f:
        disk_map = f.read().strip()
    
    # Parse files and free spaces
    files = []
    free_spaces = []
    is_file = True
    for digit in disk_map:
        if is_file:
            files.append(int(digit))
        else:
            free_spaces.append(int(digit))
        is_file = not is_file
    
    # Assign file IDs
    file_ids = list(range(len(files)))
    
    # Build the initial disk layout
    disk = []
    file_id_index = 0
    for i in range(len(files)):
        disk.extend([file_id_index] * files[i])
        if i < len(free_spaces):
            disk.extend(['.'] * free_spaces[i])
        file_id_index += 1
    
    # Compact the disk
    compacted_disk = compact_disk(disk[:])  # Use a copy to avoid modifying the original
    
    # Calculate the checksum
    checksum = calculate_checksum(compacted_disk)
    
    print("The resulting filesystem checksum is:", checksum)

if __name__ == "__main__":
    main()

The resulting filesystem checksum is: 6430446922192


## Part 2

In [5]:
def parse_disk_map(disk_map):
    files_lengths = []
    free_spaces_lengths = []
    is_file = True
    for char in disk_map:
        if is_file:
            files_lengths.append(int(char))
        else:
            free_spaces_lengths.append(int(char))
        is_file = not is_file
    return files_lengths, free_spaces_lengths

def build_initial_disk(files_lengths, free_spaces_lengths):
    disk = []
    file_positions = []
    current_pos = 0
    for i in range(len(files_lengths)):
        file_id = i
        file_length = files_lengths[i]
        file_positions.append({'id': file_id, 'start': current_pos, 'length': file_length})
        disk.extend([file_id] * file_length)
        if i < len(free_spaces_lengths):
            free_length = free_spaces_lengths[i]
            disk.extend(['.'] * free_length)
            current_pos += file_length + free_length
        else:
            current_pos += file_length
    return disk, file_positions

def find_leftmost_span(disk, file_start, file_length):
    spans = []
    start = None
    for i in range(len(disk)):
        if disk[i] == '.':
            if start is None:
                start = i
        else:
            if start is not None:
                spans.append((start, i - 1))
                start = None
    if start is not None:
        spans.append((start, len(disk) - 1))
    
    for span in spans:
        if span[1] - span[0] + 1 >= file_length and span[0] < file_start:
            return span
    return None

def move_file(disk, file_info, span_start, span_end):
    file_id = file_info['id']
    file_start = file_info['start']
    file_length = file_info['length']
    
    for i in range(file_length):
        disk[span_start + i] = file_id
        disk[file_start + i] = '.'
    
    file_info['start'] = span_start

def compact_disk(disk, file_positions):
    sorted_files = sorted(file_positions, key=lambda x: -x['id'])
    for file_info in sorted_files:
        file_start = file_info['start']
        file_length = file_info['length']
        span = find_leftmost_span(disk, file_start, file_length)
        if span:
            move_file(disk, file_info, span[0], span[1])

def calculate_checksum(disk):
    checksum = 0
    for pos, block in enumerate(disk):
        if block != '.':
            checksum += pos * block
    return checksum

def main():
    with open('input.txt', 'r') as f:
        disk_map = f.read().strip()
    
    files_lengths, free_spaces_lengths = parse_disk_map(disk_map)
    disk, file_positions = build_initial_disk(files_lengths, free_spaces_lengths)
    compact_disk(disk, file_positions)
    checksum = calculate_checksum(disk)
    print("The resulting filesystem checksum is:", checksum)

if __name__ == "__main__":
    main()

The resulting filesystem checksum is: 6460170593016
